In [1]:
import pandas as pd

### Note: it's easier for you to check the .html file in the folder before running the current script ;)

## Folium installation (just in case)

folium installation in the anaconda framework:

$ conda install -c ioos folium=0.2.1

In [2]:
import folium

## Initialization of the canton map

In [4]:
canton_map = folium.Map(
    location=[46.8182, 8.2275], 
    tiles='Mapbox Bright', 
    zoom_start=8
)
folium.Marker([46.5191, 6.5668], popup='EPFL',
                   icon = folium.Icon(icon = 'cloud')).add_to(canton_map)
folium.Marker([47.3763, 8.5477], popup='ETH',
                   icon = folium.Icon(icon = 'cloud')).add_to(canton_map)
# canton_map

### Command line magic to get the list of cantons from the json file

In [5]:
cantons =! cat "ch-cantons.topojson.json" | grep "id\":" | cut -d\" -f4

In [6]:
# convert the cantons list into a dataframe
import numpy as np
cantons = np.array(cantons)
cs = pd.DataFrame(data = cantons, index=cantons, columns=['canton'])

In [7]:
# Read the file containing the grants per each canton obtained from processing the data
grants_per_canton = pd.read_csv("data/grants_per_canton.csv", sep=';', index_col=0)
grants_per_canton.head()

,canton,Approved Amount
0,AG,1.391864e+08
1,BE,1.560234e+09
2,BL,3.476142e+06
3,BS,1.399188e+09
4,FR,4.591955e+08


### Operations to prepare the dataset for map visualization

Merge the `grants_per_canton` dataset with the dataset containing (all) the canton tags present in the json file 

In [8]:
all_cantons = cs.merge(grants_per_canton, left_on='canton', right_on='canton', how='outer')

in case there are NaN values in the table (i.e. values for some cantons are missing)


we use 1 because later we use **`log`** to rescale the axes for a better visualization


In [9]:
all_cantons = all_cantons.fillna(1)

In [10]:
all_cantons = all_cantons.set_index(['canton'])

In [11]:
all_cantons['id'] = cs['canton']

In [12]:
# we use a log scale to have more suitable values for visualization
all_cantons['Approved Amount Resized'] = np.log(all_cantons['Approved Amount'])

In [13]:
all_cantons.sort_index(inplace=True)
all_cantons.head()

,Approved Amount,id,Approved Amount Resized
canton,,,
AG,1.391864e+08,AG,18.751325
AI,1.000000e+00,AI,0.000000
AR,1.000000e+00,AR,0.000000
BE,1.560234e+09,BE,21.168102
BL,3.476142e+06,BL,15.061434


### Canton grants map visualization

In [14]:
# the threshold_scale is made to highlight the difference in grant amounts

topopath = 'ch-cantons.topojson.json'
canton_map.choropleth(
    geo_path = topopath, 
    data = all_cantons,
    columns = ['id','Approved Amount Resized'],
    key_on='feature.id',
    #key_on = 'objects.cantons.geometries.id',
    topojson = 'objects.cantons',
    fill_color='YlGn',
    threshold_scale=[1, 5, 10, 15, 20, 25],
    legend_name = 'Approved Amount',
    fill_opacity=0.7, line_opacity=0.2,
    #line_weight=2
)
canton_map

## Bonus: Rostigraben

NOTE: After extensive search we could not find any tables recording the Rostigraben division between cantons, instead we searched for the languages spoken in each canton and unfortunately we had to write them manually as (again) we could not find a downloadable table storing this information

The language(s) for each canton were added based on the information from the 
website: http://www.statoids.com/uch.html

In [15]:
rosti_cantons = all_cantons
rosti_cantons.sort_index(inplace=True)
rosti_cantons['Lang'] = ['G', 'G', 'G', 'G', 'G', 'G', 'FG', 'F', 'G', 'GR', 'F', 'G', 'F',
                         'G', 'G', 'G', 'G', 'G', 'G', 'G', 'I', 'G', 'F', 'FG', 'G', 'G']
rosti_cantons.head()

,Approved Amount,id,Approved Amount Resized,Lang
canton,,,,
AG,1.391864e+08,AG,18.751325,G
AI,1.000000e+00,AI,0.000000,G
AR,1.000000e+00,AR,0.000000,G
BE,1.560234e+09,BE,21.168102,G
BL,3.476142e+06,BL,15.061434,G


Group the cantons based on the language spoken and aggregate the grants corresponding to each spoken language(s)

In [16]:
rosti_cantons_grouped_by_lang = rosti_cantons.groupby(['Lang'])
rosti_cantons_grouped_by_lang = rosti_cantons_grouped_by_lang.agg({'Approved Amount':'sum'})
rosti_cantons_grouped_by_lang = rosti_cantons_grouped_by_lang.rename(
    columns={'Approved Amount': 'Amount per language'})
rosti_cantons_grouped_by_lang

,Amount per language
Lang,
F,4.695549e+09
FG,4.913787e+08
G,7.045919e+09
GR,3.722580e+07
I,1.563997e+08


In the next part we merge the above 2 tables such that each canton has an additional amount depending on its language, and this amount is the same for each canton that speaks the same language.

This will be helpful during the visualization because all the cantons sharing the same language(s) will have the same color.

In [17]:
rosti = rosti_cantons.merge(rosti_cantons_grouped_by_lang, 
                                  left_on='Lang', right_index=True, how='outer')
rosti['Amount per lang resized'] = np.log(rosti['Amount per language'])
rosti.head()

,Approved Amount,id,Approved Amount Resized,Lang,Amount per language,Amount per lang resized
canton,,,,,,
AG,1.391864e+08,AG,18.751325,G,7.045919e+09,22.675714
AI,1.000000e+00,AI,0.000000,G,7.045919e+09,22.675714
AR,1.000000e+00,AR,0.000000,G,7.045919e+09,22.675714
BE,1.560234e+09,BE,21.168102,G,7.045919e+09,22.675714
BL,3.476142e+06,BL,15.061434,G,7.045919e+09,22.675714


The actual visualization.

NOTE: We realize that the color gradient might suggest bigger differences to the viewer than they actually are, but we made a compromise between accuracy and visibility when creating the map to highlight the differences.

In [18]:
rosti_map = folium.Map(
    location=[46.8182, 8.2275], 
    tiles='Mapbox Bright', 
    #tiles = 'Stamen Toner',
    #tiles = 'Stamen Terrain',
    zoom_start=8
)
folium.Marker([46.5191, 6.5668], popup='EPFL',
                   icon = folium.Icon(icon = 'cloud')).add_to(rosti_map)
folium.Marker([47.3763, 8.5477], popup='ETH',
                   icon = folium.Icon(icon = 'cloud')).add_to(rosti_map)

topopath = 'ch-cantons.topojson.json'
rosti_map.choropleth(
    geo_path = topopath, 
    data = rosti,
    columns = ['id','Amount per lang resized'],
    key_on='feature.id',
    #key_on = 'objects.cantons.geometries.id',
    topojson = 'objects.cantons',
    fill_color='YlGnBu',
    threshold_scale=[15, 17, 18, 21, 22, 23],
    fill_opacity=0.7, line_opacity=0.2,
    line_weight=2
)
rosti_map